In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# ENABLE IF USING COLAB
USE_COLAB = True

if USE_COLAB:
    import os
    import shutil
    if os.path.exists('/content/spectroconv/'):
        shutil.rmtree('/content/spectroconv/') #deletes a directory and all its contents.
    !git clone https://github.com/jakeval/spectroconv.git
    !pip install hub
    !pip install hub[audio]
    !pip install wandb
    import sys
    sys.path.insert(0,'/content/spectroconv')

In [ ]:
import torch
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)


In [ ]:
from models import wb_experiment

In [ ]:
n_mels = 128

config = {
  'data': {
    'train': {
        'name': f"nsynth-full-{n_mels}-train"
    },
    'val': {
        'name': f"nsynth-full-{n_mels}-val"
    }
  },
  'group': 'lrlc',
  'job_type': 'sweep',
  'model': {
      'id': wb_experiment.ModelType.LRLC,
      'name': 'LRLC'
  },
}

# wandb's sweep config must be exact, no additional dictionary values are allowed.
wb_sweep_config = {
    'name': 'lrlc-sweep',
    'project': 'spectroconv-debug',
    'entity': 'jakeval-colab',
    'method': 'random',
    'metric': {
      'name': 'f1',
      'goal': 'maximize'
    },
    'parameters': {
      'local_dim': {
          'value': None
      },
      'kernel_size': {
          'value': 3
      },
      'num_channels': {
          'value': 32
      },
      'num_conv_layers': {
          'value': 3
      },
      'linear_layer_size': {
          'value': 1024
      },
      'learning_rate': {
          'value': 0.0001
      },
      'dropout': {# probability of an element to be zeroed (0 = no dropout)
          'value': 0.25
      },
      'l2_reg': {
          'value': 0, # trying out no l2 regularizer
          #'distribution': 'log_uniform_values',
          #'min': 0.0000001,
          #'max': 0.01,
      },
      'optimizer': {
          'value': 'adam'
      },
      'epochs': {
          'value': 2
      },
      'batch_size': {
          'value': 124
          #'distribution': 'q_log_uniform_values',
          #'q': 4,
          #'min': 72,
          #'max': 128,
      },
      'linear_reg': {
          'value': False
      },
      'rank': {
          'distribution': 'int_uniform',
          'min': 1,
          'max': 64,
      },
      'lcw': {
          'value': True
      },
      'low_dim': {
          32
      },
      'lrlc_channels': {
          'value': 128
      }
    }
}

config['sweep_config'] = wb_sweep_config

In [ ]:
sweeper = wb_experiment.SweepExperiment(wb_config=config, login_key=False)
sweeper.run_sweep(iterations = 30, save_all = False)